In [4]:
import json
import boto3
import sagemaker as sage
from sagemaker.pytorch import PyTorch
from sagemaker.estimator import Estimator

boto_session = boto3.Session()
sess = sage.Session(boto_session=boto_session)

sagemaker.config INFO - Not applying SDK defaults from location: /etc/xdg/sagemaker/config.yaml
sagemaker.config INFO - Not applying SDK defaults from location: /root/.config/sagemaker/config.yaml


In [5]:
iam = boto3.client('iam', region_name='eu-central-1')
role_name = 'flexki-nano-demo-test-role-hamza'
policy = {
    'Statement': [{
        'Action': 'sts:AssumeRole',
        'Effect': 'Allow',
        'Principal': {'Service': 'sagemaker.amazonaws.com'}},
    ],  
    'Version': '2012-10-17'
}

roles = iam.list_roles()
role_arn = None
for role in roles['Roles']:
    if role['RoleName'] == role_name:
        role_arn = role['Arn']
        
if role_arn == None:
    new_role = iam.create_role(
        AssumeRolePolicyDocument=json.dumps(policy),
        Path='/',
        RoleName=role_name,
    )
    role_arn = new_role['Role']['Arn']

    iam.attach_role_policy(
        RoleName=role_name,
        PolicyArn='arn:aws:iam::aws:policy/AmazonSageMakerFullAccess'
    )

    iam.attach_role_policy(
        RoleName=role_name,
        PolicyArn='arn:aws:iam::aws:policy/AmazonS3FullAccess'
    )

In [ ]:
estimator = PyTorch(
    entry_point='sage-entry.py',
    source_dir='./',
    role=role_arn,
    py_version='py310',
    framework_version='2.1.0',
    instance_count=1,
    instance_type='ml.g4dn.xlarge',
    env={'conda': {'environment.yml': 'environment.yml'}},
    keep_alive_period_in_seconds=300
)
estimator.fit({'train': 's3://flexki-andrii/3dgaussians/mipnerf/'})

sagemaker.config INFO - Not applying SDK defaults from location: /etc/xdg/sagemaker/config.yaml
sagemaker.config INFO - Not applying SDK defaults from location: /root/.config/sagemaker/config.yaml


INFO:sagemaker.image_uris:image_uri is not presented, retrieving image_uri based on instance_type, framework etc.


Using provided s3_resource


INFO:sagemaker.image_uris:image_uri is not presented, retrieving image_uri based on instance_type, framework etc.
INFO:sagemaker:Creating training-job with name: pytorch-training-2024-03-26-16-39-00-290


2024-03-26 16:39:12 Starting - Starting the training job...
2024-03-26 16:39:27 Starting - Preparing the instances for training......
2024-03-26 16:40:23 Downloading - Downloading input data......
2024-03-26 16:41:28 Downloading - Downloading the training image..................
2024-03-26 16:44:39 Training - Training image download completed. Training in progress...bash: cannot set terminal process group (-1): Inappropriate ioctl for device
bash: no job control in this shell
2024-03-26 16:44:51,989 sagemaker-training-toolkit INFO     Imported framework sagemaker_pytorch_container.training
2024-03-26 16:44:52,015 sagemaker-training-toolkit INFO     No Neurons detected (normal if no neurons installed)
2024-03-26 16:44:52,025 sagemaker_pytorch_container.training INFO     Block until all host DNS lookups succeed.
2024-03-26 16:44:52,039 sagemaker_pytorch_container.training INFO     Invoking user training script.
2024-03-26 16:44:55,374 sagemaker-training-toolkit INFO     No Neurons detect